# Verifying ZZ Measurement Circuits with Stabilizer Simulation

This notebook demonstrates how to verify the correctness of different circuit implementations 
for measuring the two-qubit Pauli ZZ operator using stabilizer simulation.

## Why ZZ Measurements Matter

In quantum error correction, measuring multi-qubit Pauli operators like ZZ is fundamental. 
These measurements extract syndrome information without destroying the encoded quantum state. 
However, ZZ is not a directly measurable observable on most hardware—we need circuits that 
implement it using available gates.

## Our Verification Approach

We use the **Choi-Jamiołkowski isomorphism**: instead of testing a circuit on all possible 
input states, we apply it to half of maximally entangled Bell pairs. If the output stabilizers 
match those of the ideal operation, the circuit is correct for *all* inputs. 
We use `OutcomeCompleteSimulation` to check correctness for *all* measurement outcomes.

## Setup

First, we import the stabilizer simulation tools from `paulimer`:

In [ ]:
from paulimer import OutcomeCompleteSimulation
from paulimer import UnitaryOpcode
from paulimer import SparsePauli

### Primitive Operations

We define helper functions for the basic quantum operations we'll use:

- **Hadamard (H)** and **CNOT (CX)** gates
- **Bell pair preparation** - creates the entangled state $(|00\rangle + |11\rangle)/\sqrt{2}$ from $|00\rangle$
- **Measure-and-reset** - measures a qubit in the Z basis and resets it to $|0\rangle$
- **Pauli operators** - ZZ and XX for defining stabilizers

In [ ]:
def h(sim, target):
    sim.apply_unitary(unitary_op= UnitaryOpcode.Hadamard, support=[target])

def cx(sim, control, target):
    sim.apply_unitary(unitary_op= UnitaryOpcode.ControlledX, support=[control, target])

def prepare_bell_pair(sim, qubit_a, qubit_b):
    sim.apply_unitary(unitary_op= UnitaryOpcode.PrepareBell, support=[qubit_a, qubit_b])

def prepare_bell_pairs(sim, qubits_a, qubits_b):
    for (a,b) in zip(qubits_a,qubits_b):
        prepare_bell_pair(sim, a,b)

def m_reset_z(sim, target):
    bit = sim.measure(SparsePauli.z(target))
    sim.apply_conditional_pauli(SparsePauli.x(target), [bit])
    return bit

def zz(qubit_a, qubit_b):
    return SparsePauli.z(qubit_a) * SparsePauli.z(qubit_b)

def measure_zz(sim, qubit_a, qubit_b):
    bit = sim.measure(zz(qubit_a, qubit_b))
    return bit

def xx(qubit_a, qubit_b):
    return SparsePauli.x(qubit_a) * SparsePauli.x(qubit_b)


## Circuit 1: ZZ Measurement via Two CNOTs

In [ ]:
def measure_zz_via_cnot(sim, qubit_a, qubit_b, aux):
    """Measure ZZ using two CNOTs and a single auxiliary qubit."""
    cx(sim, qubit_a, aux)
    cx(sim, qubit_b, aux)
    bit = m_reset_z(sim, aux)
    return [bit]


def test_measure_zz_via_cnot():
    sim = OutcomeCompleteSimulation()
    (ref_a, ref_b, qubit_a, qubit_b, aux) = range(5)
    
    # Prepare input: halves of two Bell pairs
    prepare_bell_pairs(sim, [ref_a, ref_b], [qubit_a, qubit_b])
    
    # Apply the circuit under test
    outcome = measure_zz_via_cnot(sim, qubit_a, qubit_b, aux)
    
    # Verify: auxiliary qubit is reset to |0⟩
    assert sim.is_stabilizer(SparsePauli.z(aux))
    
    # Verify: output is in a ZZ eigenstate (stabilized by ±ZZ)
    assert sim.is_stabilizer(zz(qubit_a, qubit_b), ignore_sign=True)
    
    # Verify: the ZZ eigenvalue matches the measurement outcome
    assert sim.is_stabilizer(zz(qubit_a, qubit_b), sign_parity=outcome)

In [ ]:
test_measure_zz_via_cnot()
print("✓ Circuit 1 (CNOT-based) passed verification")

## Circuit 2: ZZ Measurement via Bell Pair

In [ ]:
def measure_zz_via_bell(sim, qubit_a, qubit_b, aux_a, aux_b):
    """Measure ZZ using an auxiliary Bell pair for fault tolerance."""
    prepare_bell_pair(sim, aux_a, aux_b)
    cx(sim, qubit_a, aux_a)
    cx(sim, qubit_b, aux_b)
    bit_a = m_reset_z(sim, aux_a)
    bit_b = m_reset_z(sim, aux_b)
    return [bit_a, bit_b]  # XOR of these bits gives ZZ eigenvalue


def test_measure_zz_via_bell():
    sim = OutcomeCompleteSimulation()
    (ref_a, ref_b, qubit_a, qubit_b, aux_a, aux_b) = range(6)
    
    # Prepare input: halves of two Bell pairs
    prepare_bell_pairs(sim, [ref_a, ref_b], [qubit_a, qubit_b])
    
    # Apply the circuit under test
    outcome = measure_zz_via_bell(sim, qubit_a, qubit_b, aux_a, aux_b)
    
    # Verify: both auxiliary qubits are reset to |0⟩
    assert sim.is_stabilizer(SparsePauli.z(aux_a))
    assert sim.is_stabilizer(SparsePauli.z(aux_b))
    
    # Verify: output is in a ZZ eigenstate
    assert sim.is_stabilizer(zz(qubit_a, qubit_b), ignore_sign=True)
    
    # Verify: ZZ eigenvalue matches the parity of both measurement outcomes
    assert sim.is_stabilizer(zz(qubit_a, qubit_b), sign_parity=outcome)

In [ ]:
test_measure_zz_via_bell()
print("✓ Circuit 2 (Bell-based) passed verification")

## Rigorous Verification: Checking the Full Choi State

The tests above check basic properties. For complete verification, we compare against the 
**ideal ZZ measurement** and check that all stabilizers of the Choi state match.

The ideal Choi state after a ZZ measurement should satisfy:
- $Z_{\text{ref}_a} Z_{\text{ref}_b}$ with sign = measurement outcome ( ZZ observable is measured )
- $Z_{\text{qubit}_a} Z_{\text{qubit}_b}$ with sign = measurement outcome (output is ZZ eigenstate with the sign given my the outcome)
- $Z_{\text{ref}_a} Z_{\text{qubit}_a}$ with sign +1 (Z observable preserved as is)
- $X_{\text{ref}_a} X_{\text{ref}_b} X_{\text{qubit}_a} X_{\text{qubit}_b}$ with sign +1 (XX observable preserved as is)

In [ ]:
def assert_zz_measure_choi_state(sim, ref_a, ref_b, qubit_a, qubit_b, parity):
    assert sim.is_stabilizer(zz(ref_a,ref_b), sign_parity=parity)
    assert sim.is_stabilizer(zz(qubit_a,qubit_b), sign_parity=parity)
    assert sim.is_stabilizer(zz(ref_a,qubit_a))
    assert sim.is_stabilizer(xx(ref_a,ref_b)*xx(qubit_a,qubit_b))

def test_zz_choi_stabilizers():
    sim = OutcomeCompleteSimulation()
    (ref_a, ref_b, qubit_a, qubit_b) = range(4)
    prepare_bell_pairs(sim,[ref_a,ref_b],[qubit_a,qubit_b])
    bit = measure_zz(sim,qubit_a,qubit_b)
    assert_zz_measure_choi_state(sim,ref_a, ref_b, qubit_a, qubit_b, [bit])

        

### Reference: Ideal ZZ Measurement

First, let's verify that our reference implementation (direct Pauli measurement) produces 
the correct Choi state:

In [ ]:
test_zz_choi_stabilizers()

### Full Verification of Circuit 1 (CNOT-based)

In [ ]:
def verify_zz_measurement_1():
    sim = OutcomeCompleteSimulation()
    # qubits ids for the test
    (ref_a, ref_b, qubit_a, qubit_b, aux ) = range(5)
    prepare_bell_pairs(sim,[ref_a,ref_b],[qubit_a,qubit_b])
    outcome = measure_zz_via_cnot(sim, qubit_a, qubit_b, aux)
    assert sim.is_stabilizer(SparsePauli.z(aux)) # aux is reset to |0> state
    assert_zz_measure_choi_state(sim,ref_a, ref_b, qubit_a, qubit_b, outcome)

### Full Verification of Circuit 2 (Bell-based)

In [ ]:
def verify_zz_measurement_2():
    sim = OutcomeCompleteSimulation()
    # qubits ids for the test
    (ref_a, ref_b, qubit_a, qubit_b, aux ) = range(5)
    prepare_bell_pairs(sim,[ref_a,ref_b],[qubit_a,qubit_b])
    outcome = measure_zz_via_cnot(sim, qubit_a, qubit_b, aux)
    assert sim.is_stabilizer(SparsePauli.z(aux)) # aux is reset to |0> state
    assert_zz_measure_choi_state(sim,ref_a, ref_b, qubit_a, qubit_b, outcome)